![logo](logo/DaftCode_logo_854x210.jpg)

# Sci-kit learn: pierwsze kroki

In [1]:
import sklearn

In [2]:
print(sklearn.__doc__)


Machine learning module for Python

sklearn is a Python module integrating classical machine
learning algorithms in the tightly-knit world of scientific Python
packages (numpy, scipy, matplotlib).

It aims to provide simple and efficient solutions to learning problems
that are accessible to everybody and reusable in various contexts:
machine-learning as a versatile tool for science and engineering.

See http://scikit-learn.org for complete documentation.



In [3]:
sklearn.__version__

'0.18.1'

# Przykład zaczerpnięty z: http://scikit-learn.org/stable/auto_examples/missing_values.html

In [4]:
from sklearn.datasets import load_boston

In [5]:
dataset = sklearn.datasets.load_boston()

In [12]:
from sklearn.datasets.base import Bunch

In [ ]:
import pandas as pd

DF_data = pd.DataFrame(dataset.data, columns=dataset.feature_names)

In [68]:
DF_data.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.593761,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063
std,8.596783,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000
75%,3.647423,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000


In [50]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [70]:
X_full, y_full = dataset.data, dataset.target

n_samples = X_full.shape[0]
n_features = X_full.shape[1]

estimator = LinearRegression()

score = cross_val_score(estimator, X_full, y_full, scoring='neg_mean_squared_error', cv=10).mean()
print("Mean score = {}".format(score))

Mean score = -34.7630915054


In [71]:
from sklearn.ensemble import RandomForestRegressor

In [72]:
estimator = RandomForestRegressor(n_estimators=100)
score = cross_val_score(estimator, X_full, y_full, scoring='neg_mean_squared_error', cv=10).mean()
print("Mean score = {}".format(score))

Mean score = -22.0575713262


In [79]:
import numpy as np

rng = np.random.RandomState(0)

missing_rate = 0.75
n_missing_samples = np.floor(n_samples * missing_rate)
missing_samples = np.hstack((np.zeros(n_samples - n_missing_samples,
                                      dtype=np.bool),
                             np.ones(n_missing_samples,
                                     dtype=np.bool)))
rng.shuffle(missing_samples)
missing_features = rng.randint(0, n_features, n_missing_samples)

X_filtered = X_full[~missing_samples, :]
y_filtered = y_full[~missing_samples]

estimator = RandomForestRegressor(n_estimators=100)
score = cross_val_score(estimator, X_filtered, y_filtered, scoring='neg_mean_squared_error', cv=10).mean()
print("Mean score = {}".format(score))

/home/md/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:8: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/md/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:12: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Mean score = -28.2297257686


In [76]:
from sklearn.preprocessing import Imputer

In [78]:
X_missing = X_full.copy()
X_missing[np.where(missing_samples)[0], missing_features] = 0
y_missing = y_full.copy()

imputer = Imputer(missing_values=0, strategy="mean", axis=0)
imputer.fit(X_missing)

X_filled_out = imputer.transform(X_missing)
estimator = RandomForestRegressor(n_estimators=100)
score = cross_val_score(estimator, X_filled_out, y_missing, scoring='neg_mean_squared_error', cv=10).mean()
print("Mean score = {}".format(score))

Mean score = -22.8833819907


In [ ]:
estimator = Pipeline([("imputer", Imputer(missing_values=0, strategy="mean", axis=0)),
                      ("forest", RandomForestRegressor(n_estimators=100))])
score = cross_val_score(estimator, X_missing, y_missing).mean()
print("Mean score = {}".format(score))